In [ ]:
import json

import networkx as nx

## Introduction

This notebook is built as an example for how BOMs can be converted into graphs. It is not meant to exhaustively support all BOMs, but to provide a template for how to convert BOMs into graphs. The goal being to reduce the burden of getting started analyzing BOMS with the graph_merge algorithm.

It should be noted that the graph_merge algorithm does not currently handle sparsity and all nodes must contain the same attributes. Similarity if the goal is to compare across different SBOM standards, it will fall on the user to determine the appropriate field mappings and adjust the node/edge attribute names accordingly.

## CycloneDX v1.x

CycloneDX keeps most of its interesting data sitting inside of `component` objects.
The relationships between components can either be a "parent-child" type relationship or dependency related.
The "parent-child" relationship is implicitly declared using a recursive data structure.
Every `component` object has an optional `components` property that lists all children `component`s of it.
The dependency relationships are defined explicitly in the `dependencies` property.

The code below converts a CycloneDX BOM into a graph by first flattening the `component` objects structure and converting them to nodes.
The different relationships are then added as edges with an edge attribute detailing the exact relationship type.
A minimal set of node attributes are captured, although more could be captured by modifying the `convert_cyclonedx_component_to_node` function.
If you intend to use this graph with the `graph_merge` package then please remember that it does not handle node attribute sparsity.
All nodes must contain the same set of attributes.


In [ ]:
def convert_cyclonedx_component_to_node(component: dict) -> tuple[int, dict]:
    # change this part to add more or less node attributes
    node_attrs = {
        "bom-ref": component.get("bom-ref", ""),
        "name": component.get("name", ""),
        "type": component.get("type", ""),
        "version": component.get("version", ""),
    }

    return (str(component["$id"]), node_attrs)


def flatten_cyclonedx_components(components: list[dict], parent_id: int, next_id: int) -> list[dict]:
    # cyclonedx has a recursive component structure of parent-child relationships
    # that will need to get flattened out
    # cyclonedx components are not required to have a unique identity, so we also
    # assign a numeric node ID to ensure we can represent the components uniquely in
    # the final graph, that numeric node ID will be stored in a new key called $id

    result = []
    for component in components:
        component = component.copy()  # noqa: PLW2901
        component["$id"] = next_id
        component["$parent_id"] = parent_id

        subcomponents = flatten_cyclonedx_components(component.get("components", []), next_id, next_id + 1)

        next_id += len(subcomponents) + 1
        result.append(component)
        result.extend(subcomponents)
    return result


def generate_cyclonedx_dependency_edges(
    dependencies: list[dict], components: list[dict]
) -> list[tuple[int, int, dict]]:
    ref_lookup = {c["bom-ref"]: c["$id"] for c in components}

    edges = []
    for dependency in dependencies:
        ref = ref_lookup.get(dependency.get("ref"))
        dependency_refs = dependency.get("dependsOn", [])
        if ref is not None and dependency_refs:
            for depends_on in dependency_refs:
                depends_on = ref_lookup.get(depends_on)  # noqa: PLW2901
                if depends_on:
                    edges.append((str(ref), str(depends_on), {"relationship": "dependsOn"}))
                else:
                    print(f"dependsOn could not be found: {depends_on}")
        elif ref is None:
            print(f"Ref could not be located: {ref}")

    return edges


def parse_cyclonedx_bom(bom: dict) -> nx.Graph:
    # the actual software being described by the bom is supposed to be stored in the metadata section
    components = flatten_cyclonedx_components(filter(lambda x: x, [bom.get("metadata", {}).get("component")]), None, 0)

    # the components section is a general list of software components that will be referenced in the document
    components.extend(flatten_cyclonedx_components(bom.get("components", []), None, len(components)))

    nodes = [convert_cyclonedx_component_to_node(c) for c in components]
    edges = [
        (str(c["$parent_id"]), str(c["$id"]), {"relationship": "parentOf"}) for c in components if c.get("$parent_id")
    ]

    edges.extend(generate_cyclonedx_dependency_edges(bom.get("dependencies", []), components))

    g = nx.Graph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)

    return g

In [ ]:
# heavily reduced version of https://github.com/CycloneDX/bom-examples/blob/master/SBOM/protonmail-webclient-v4-0912dff/bom.json
protonmail_bom = r"""
{
  "bomFormat": "CycloneDX",
  "specVersion": "1.2",
  "serialNumber": "urn:uuid:2392d49c-ea93-44e0-aa36-5923fcfb5efb",
  "version": 1,
  "metadata": {
    "timestamp": "2021-05-16T17:08:44+02:00",
    "component": {
      "bom-ref": "pkg:golang/github.com/ProtonMail/proton-bridge@v1.6.3",
      "type": "application",
      "name": "github.com/ProtonMail/proton-bridge",
      "version": "v1.6.3"
    }
  },
  "components": [
    {
      "bom-ref": "pkg:golang/github.com/CloudyKit/jet/v3@v3.0.0",
      "type": "library",
      "name": "github.com/CloudyKit/jet/v3",
      "version": "v3.0.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/ProtonMail/go-crypto@v0.0.0-20201208171014-cdb7591792e2",
      "type": "library",
      "name": "github.com/ProtonMail/go-crypto",
      "version": "v0.0.0-20201208171014-cdb7591792e2"
    },
    {
      "bom-ref": "pkg:golang/github.com/ProtonMail/go-rfc5322@v0.5.0",
      "type": "library",
      "name": "github.com/ProtonMail/go-rfc5322",
      "version": "v0.5.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/ProtonMail/go-vcard@v0.0.0-20180326232728-33aaa0a0c8a5",
      "type": "library",
      "name": "github.com/ProtonMail/go-vcard",
      "version": "v0.0.0-20180326232728-33aaa0a0c8a5"
    },
    {
      "bom-ref": "pkg:golang/github.com/ProtonMail/gopenpgp/v2@v2.1.3",
      "type": "library",
      "name": "github.com/ProtonMail/gopenpgp/v2",
      "version": "v2.1.3"
    },
    {
      "bom-ref": "pkg:golang/github.com/PuerkitoBio/goquery@v1.5.1",
      "type": "library",
      "name": "github.com/PuerkitoBio/goquery",
      "version": "v1.5.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/cpuguy83/go-md2man@v1.0.10",
      "type": "library",
      "name": "github.com/cpuguy83/go-md2man",
      "version": "v1.0.10"
    },
    {
      "bom-ref": "pkg:golang/github.com/cpuguy83/go-md2man/v2@v2.0.0-20190314233015-f79a8a8ca69d",
      "type": "library",
      "name": "github.com/cpuguy83/go-md2man/v2",
      "version": "v2.0.0-20190314233015-f79a8a8ca69d"
    },
    {
      "bom-ref": "pkg:golang/github.com/dgraph-io/badger@v1.6.0",
      "type": "library",
      "name": "github.com/dgraph-io/badger",
      "version": "v1.6.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/ProtonMail/docker-credential-helpers@v1.1.0",
      "type": "library",
      "name": "github.com/ProtonMail/docker-credential-helpers",
      "version": "v1.1.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/emersion/go-message@v0.12.1-0.20201221184100-40c3f864532b",
      "type": "library",
      "name": "github.com/emersion/go-message",
      "version": "v0.12.1-0.20201221184100-40c3f864532b"
    },
    {
      "bom-ref": "pkg:golang/github.com/emersion/go-smtp@v0.14.0",
      "type": "library",
      "name": "github.com/emersion/go-smtp",
      "version": "v0.14.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/fatih/color@v1.9.0",
      "type": "library",
      "name": "github.com/fatih/color",
      "version": "v1.9.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/getsentry/sentry-go@v0.8.0",
      "type": "library",
      "name": "github.com/getsentry/sentry-go",
      "version": "v0.8.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/gin-gonic/gin@v1.4.0",
      "type": "library",
      "name": "github.com/gin-gonic/gin",
      "version": "v1.4.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/google/go-cmp@v0.5.1",
      "type": "library",
      "name": "github.com/google/go-cmp",
      "version": "v0.5.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/hashicorp/go-multierror@v1.1.0",
      "type": "library",
      "name": "github.com/hashicorp/go-multierror",
      "version": "v1.1.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/hashicorp/go-version@v1.2.0",
      "type": "library",
      "name": "github.com/hashicorp/go-version",
      "version": "v1.2.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/hashicorp/hcl@v1.0.0",
      "type": "library",
      "name": "github.com/hashicorp/hcl",
      "version": "v1.0.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/iris-contrib/jade@v1.1.3",
      "type": "library",
      "name": "github.com/iris-contrib/jade",
      "version": "v1.1.3"
    },
    {
      "bom-ref": "pkg:golang/github.com/iris-contrib/pongo2@v0.0.1",
      "type": "library",
      "name": "github.com/iris-contrib/pongo2",
      "version": "v0.0.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/json-iterator/go@v1.1.9",
      "type": "library",
      "name": "github.com/json-iterator/go",
      "version": "v1.1.9"
    },
    {
      "bom-ref": "pkg:golang/github.com/kataras/golog@v0.0.10",
      "type": "library",
      "name": "github.com/kataras/golog",
      "version": "v0.0.10"
    },
    {
      "bom-ref": "pkg:golang/github.com/kataras/iris/v12@v12.1.8",
      "type": "library",
      "name": "github.com/kataras/iris/v12",
      "version": "v12.1.8"
    },
    {
      "bom-ref": "pkg:golang/github.com/kataras/neffos@v0.0.14",
      "type": "library",
      "name": "github.com/kataras/neffos",
      "version": "v0.0.14"
    },
    {
      "bom-ref": "pkg:golang/github.com/keybase/go-keychain@v0.0.0-20200502122510-cda31fe0c86d",
      "type": "library",
      "name": "github.com/keybase/go-keychain",
      "version": "v0.0.0-20200502122510-cda31fe0c86d"
    },
    {
      "bom-ref": "pkg:golang/github.com/kr/text@v0.2.0",
      "type": "library",
      "name": "github.com/kr/text",
      "version": "v0.2.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/labstack/echo/v4@v4.1.11",
      "type": "library",
      "name": "github.com/labstack/echo/v4",
      "version": "v4.1.11"
    },
    {
      "bom-ref": "pkg:golang/github.com/labstack/gommon@v0.3.0",
      "type": "library",
      "name": "github.com/labstack/gommon",
      "version": "v0.3.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/mediocregopher/radix/v3@v3.4.2",
      "type": "library",
      "name": "github.com/mediocregopher/radix/v3",
      "version": "v3.4.2"
    },
    {
      "bom-ref": "pkg:golang/github.com/nats-io/jwt@v0.3.0",
      "type": "library",
      "name": "github.com/nats-io/jwt",
      "version": "v0.3.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/nats-io/nats.go@v1.9.1",
      "type": "library",
      "name": "github.com/nats-io/nats.go",
      "version": "v1.9.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/onsi/gomega@v1.7.1",
      "type": "library",
      "name": "github.com/onsi/gomega",
      "version": "v1.7.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/sirupsen/logrus@v1.7.0",
      "type": "library",
      "name": "github.com/sirupsen/logrus",
      "version": "v1.7.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/smartystreets/goconvey@v1.6.4",
      "type": "library",
      "name": "github.com/smartystreets/goconvey",
      "version": "v1.6.4"
    },
    {
      "bom-ref": "pkg:golang/github.com/spf13/afero@v1.1.2",
      "type": "library",
      "name": "github.com/spf13/afero",
      "version": "v1.1.2"
    },
    {
      "bom-ref": "pkg:golang/github.com/spf13/cast@v1.3.0",
      "type": "library",
      "name": "github.com/spf13/cast",
      "version": "v1.3.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/spf13/cobra@v0.0.5",
      "type": "library",
      "name": "github.com/spf13/cobra",
      "version": "v0.0.5"
    },
    {
      "bom-ref": "pkg:golang/github.com/spf13/viper@v1.3.2",
      "type": "library",
      "name": "github.com/spf13/viper",
      "version": "v1.3.2"
    },
    {
      "bom-ref": "pkg:golang/github.com/stretchr/objx@v0.2.0",
      "type": "library",
      "name": "github.com/stretchr/objx",
      "version": "v0.2.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/stretchr/testify@v1.6.1",
      "type": "library",
      "name": "github.com/stretchr/testify",
      "version": "v1.6.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/therecipe/qt@v0.0.0-20200701200531-7f61353ee73e",
      "type": "library",
      "name": "github.com/therecipe/qt",
      "version": "v0.0.0-20200701200531-7f61353ee73e"
    },
    {
      "bom-ref": "pkg:golang/github.com/ugorji/go@v1.1.7",
      "type": "library",
      "name": "github.com/ugorji/go",
      "version": "v1.1.7"
    },
    {
      "bom-ref": "pkg:golang/github.com/ugorji/go/codec@v1.1.7",
      "type": "library",
      "name": "github.com/ugorji/go/codec",
      "version": "v1.1.7"
    },
    {
      "bom-ref": "pkg:golang/github.com/urfave/cli/v2@v2.2.0",
      "type": "library",
      "name": "github.com/urfave/cli/v2",
      "version": "v2.2.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/valyala/fasthttp@v1.6.0",
      "type": "library",
      "name": "github.com/valyala/fasthttp",
      "version": "v1.6.0"
    },
    {
      "bom-ref": "pkg:golang/github.com/valyala/fasttemplate@v1.0.1",
      "type": "library",
      "name": "github.com/valyala/fasttemplate",
      "version": "v1.0.1"
    },
    {
      "bom-ref": "pkg:golang/github.com/vmihailenco/msgpack/v5@v5.1.3",
      "type": "library",
      "name": "github.com/vmihailenco/msgpack/v5",
      "version": "v5.1.3"
    },
    {
      "bom-ref": "pkg:golang/github.com/xeipuuv/gojsonschema@v1.2.0",
      "type": "library",
      "name": "github.com/xeipuuv/gojsonschema",
      "version": "v1.2.0"
    },
    {
      "bom-ref": "pkg:golang/golang.org/x/exp@v0.0.0-20190731235908-ec7cb31e5a56",
      "type": "library",
      "name": "golang.org/x/exp",
      "version": "v0.0.0-20190731235908-ec7cb31e5a56"
    },
    {
      "bom-ref": "pkg:golang/golang.org/x/mobile@v0.0.0-20200801112145-973feb4309de",
      "type": "library",
      "name": "golang.org/x/mobile",
      "version": "v0.0.0-20200801112145-973feb4309de"
    },
    {
      "bom-ref": "pkg:golang/golang.org/x/net@v0.0.0-20200707034311-ab3426394381",
      "type": "library",
      "name": "golang.org/x/net",
      "version": "v0.0.0-20200707034311-ab3426394381"
    }
  ],
  "dependencies": [
    {
      "ref": "pkg:golang/github.com/CloudyKit/jet/v3@v3.0.0"
    },
    {
      "ref": "pkg:golang/github.com/ProtonMail/go-crypto@v0.0.0-20201208171014-cdb7591792e2"
    },
    {
      "ref": "pkg:golang/github.com/ProtonMail/go-rfc5322@v0.5.0",
      "dependsOn": [
        "pkg:golang/github.com/sirupsen/logrus@v1.7.0",
        "pkg:golang/github.com/stretchr/testify@v1.6.1"
      ]
    },
    {
      "ref": "pkg:golang/github.com/ProtonMail/go-vcard@v0.0.0-20180326232728-33aaa0a0c8a5"
    },
    {
      "ref": "pkg:golang/github.com/ProtonMail/gopenpgp/v2@v2.1.3",
      "dependsOn": [
        "pkg:golang/github.com/ProtonMail/go-crypto@v0.0.0-20201208171014-cdb7591792e2",
        "pkg:golang/golang.org/x/mobile@v0.0.0-20200801112145-973feb4309de"
      ]
    },
    {
      "ref": "pkg:golang/github.com/PuerkitoBio/goquery@v1.5.1"
    },
    {
      "ref": "pkg:golang/github.com/cpuguy83/go-md2man@v1.0.10"
    },
    {
      "ref": "pkg:golang/github.com/cpuguy83/go-md2man/v2@v2.0.0-20190314233015-f79a8a8ca69d"
    },
    {
      "ref": "pkg:golang/github.com/dgraph-io/badger@v1.6.0",
      "dependsOn": [
        "pkg:golang/github.com/spf13/cobra@v0.0.5"
      ]
    },
    {
      "ref": "pkg:golang/github.com/ProtonMail/docker-credential-helpers@v1.1.0"
    },
    {
      "ref": "pkg:golang/github.com/emersion/go-message@v0.12.1-0.20201221184100-40c3f864532b"
    },
    {
      "ref": "pkg:golang/github.com/emersion/go-smtp@v0.14.0"
    },
    {
      "ref": "pkg:golang/github.com/fatih/color@v1.9.0"
    },
    {
      "ref": "pkg:golang/github.com/getsentry/sentry-go@v0.8.0",
      "dependsOn": [
        "pkg:golang/github.com/gin-gonic/gin@v1.4.0",
        "pkg:golang/github.com/kataras/iris/v12@v12.1.8",
        "pkg:golang/github.com/labstack/echo/v4@v4.1.11",
        "pkg:golang/github.com/onsi/gomega@v1.7.1",
        "pkg:golang/github.com/smartystreets/goconvey@v1.6.4",
        "pkg:golang/github.com/ugorji/go@v1.1.7",
        "pkg:golang/github.com/valyala/fasthttp@v1.6.0",
        "pkg:golang/github.com/xeipuuv/gojsonschema@v1.2.0"
      ]
    },
    {
      "ref": "pkg:golang/github.com/gin-gonic/gin@v1.4.0"
    },
    {
      "ref": "pkg:golang/github.com/google/go-cmp@v0.5.1"
    },
    {
      "ref": "pkg:golang/github.com/hashicorp/go-multierror@v1.1.0"
    },
    {
      "ref": "pkg:golang/github.com/hashicorp/go-version@v1.2.0"
    },
    {
      "ref": "pkg:golang/github.com/hashicorp/hcl@v1.0.0"
    },
    {
      "ref": "pkg:golang/github.com/iris-contrib/jade@v1.1.3"
    },
    {
      "ref": "pkg:golang/github.com/iris-contrib/pongo2@v0.0.1"
    },
    {
      "ref": "pkg:golang/github.com/json-iterator/go@v1.1.9"
    },
    {
      "ref": "pkg:golang/github.com/kataras/golog@v0.0.10"
    },
    {
      "ref": "pkg:golang/github.com/kataras/iris/v12@v12.1.8",
      "dependsOn": [
        "pkg:golang/github.com/CloudyKit/jet/v3@v3.0.0",
        "pkg:golang/github.com/dgraph-io/badger@v1.6.0",
        "pkg:golang/github.com/hashicorp/go-version@v1.2.0",
        "pkg:golang/github.com/iris-contrib/pongo2@v0.0.1",
        "pkg:golang/github.com/iris-contrib/jade@v1.1.3",
        "pkg:golang/github.com/json-iterator/go@v1.1.9",
        "pkg:golang/github.com/kataras/golog@v0.0.10",
        "pkg:golang/github.com/kataras/neffos@v0.0.14",
        "pkg:golang/github.com/mediocregopher/radix/v3@v3.4.2"
      ]
    },
    {
      "ref": "pkg:golang/github.com/kataras/neffos@v0.0.14",
      "dependsOn": [
        "pkg:golang/github.com/mediocregopher/radix/v3@v3.4.2",
        "pkg:golang/github.com/nats-io/nats.go@v1.9.1"
      ]
    },
    {
      "ref": "pkg:golang/github.com/keybase/go-keychain@v0.0.0-20200502122510-cda31fe0c86d",
      "dependsOn": [
        "pkg:golang/github.com/kr/text@v0.2.0"
      ]
    },
    {
      "ref": "pkg:golang/github.com/kr/text@v0.2.0"
    },
    {
      "ref": "pkg:golang/github.com/labstack/echo/v4@v4.1.11",
      "dependsOn": [
        "pkg:golang/github.com/labstack/gommon@v0.3.0",
        "pkg:golang/github.com/valyala/fasttemplate@v1.0.1"
      ]
    },
    {
      "ref": "pkg:golang/github.com/labstack/gommon@v0.3.0",
      "dependsOn": [
        "pkg:golang/github.com/valyala/fasttemplate@v1.0.1"
      ]
    },
    {
      "ref": "pkg:golang/github.com/mediocregopher/radix/v3@v3.4.2"
    },
    {
      "ref": "pkg:golang/github.com/nats-io/jwt@v0.3.0"
    },
    {
      "ref": "pkg:golang/github.com/nats-io/nats.go@v1.9.1",
      "dependsOn": [
        "pkg:golang/github.com/nats-io/jwt@v0.3.0"
      ]
    },
    {
      "ref": "pkg:golang/github.com/onsi/gomega@v1.7.1"
    },
    {
      "ref": "pkg:golang/github.com/sirupsen/logrus@v1.7.0"
    },
    {
      "ref": "pkg:golang/github.com/smartystreets/goconvey@v1.6.4"
    },
    {
      "ref": "pkg:golang/github.com/spf13/afero@v1.1.2"
    },
    {
      "ref": "pkg:golang/github.com/spf13/cast@v1.3.0"
    },
    {
      "ref": "pkg:golang/github.com/spf13/cobra@v0.0.5",
      "dependsOn": [
        "pkg:golang/github.com/cpuguy83/go-md2man@v1.0.10",
        "pkg:golang/github.com/spf13/viper@v1.3.2"
      ]
    },
    {
      "ref": "pkg:golang/github.com/spf13/viper@v1.3.2",
      "dependsOn": [
        "pkg:golang/github.com/hashicorp/hcl@v1.0.0",
        "pkg:golang/github.com/spf13/afero@v1.1.2",
        "pkg:golang/github.com/spf13/cast@v1.3.0"
      ]
    },
    {
      "ref": "pkg:golang/github.com/stretchr/objx@v0.2.0"
    },
    {
      "ref": "pkg:golang/github.com/stretchr/testify@v1.6.1"
    },
    {
      "ref": "pkg:golang/github.com/therecipe/qt@v0.0.0-20200701200531-7f61353ee73e",
      "dependsOn": [
        "pkg:golang/github.com/stretchr/objx@v0.2.0"
      ]
    },
    {
      "ref": "pkg:golang/github.com/ugorji/go@v1.1.7",
      "dependsOn": [
        "pkg:golang/github.com/ugorji/go/codec@v1.1.7"
      ]
    },
    {
      "ref": "pkg:golang/github.com/ugorji/go/codec@v1.1.7",
      "dependsOn": [
        "pkg:golang/github.com/ugorji/go@v1.1.7"
      ]
    },
    {
      "ref": "pkg:golang/github.com/urfave/cli/v2@v2.2.0",
      "dependsOn": [
        "pkg:golang/github.com/cpuguy83/go-md2man/v2@v2.0.0-20190314233015-f79a8a8ca69d"
      ]
    },
    {
      "ref": "pkg:golang/github.com/valyala/fasthttp@v1.6.0"
    },
    {
      "ref": "pkg:golang/github.com/valyala/fasttemplate@v1.0.1"
    },
    {
      "ref": "pkg:golang/github.com/vmihailenco/msgpack/v5@v5.1.3",
      "dependsOn": [
        "pkg:golang/github.com/stretchr/testify@v1.6.1"
      ]
    },
    {
      "ref": "pkg:golang/github.com/xeipuuv/gojsonschema@v1.2.0"
    },
    {
      "ref": "pkg:golang/golang.org/x/exp@v0.0.0-20190731235908-ec7cb31e5a56"
    },
    {
      "ref": "pkg:golang/golang.org/x/mobile@v0.0.0-20200801112145-973feb4309de",
      "dependsOn": [
        "pkg:golang/golang.org/x/exp@v0.0.0-20190731235908-ec7cb31e5a56"
      ]
    },
    {
      "ref": "pkg:golang/golang.org/x/net@v0.0.0-20200707034311-ab3426394381"
    },
    {
      "ref": "pkg:golang/github.com/ProtonMail/proton-bridge@v1.6.3",
      "dependsOn": [
        "pkg:golang/github.com/ProtonMail/go-rfc5322@v0.5.0",
        "pkg:golang/github.com/ProtonMail/go-vcard@v0.0.0-20180326232728-33aaa0a0c8a5",
        "pkg:golang/github.com/ProtonMail/gopenpgp/v2@v2.1.3",
        "pkg:golang/github.com/PuerkitoBio/goquery@v1.5.1",
        "pkg:golang/github.com/ProtonMail/docker-credential-helpers@v1.1.0",
        "pkg:golang/github.com/emersion/go-message@v0.12.1-0.20201221184100-40c3f864532b",
        "pkg:golang/github.com/emersion/go-smtp@v0.14.0",
        "pkg:golang/github.com/fatih/color@v1.9.0",
        "pkg:golang/github.com/getsentry/sentry-go@v0.8.0",
        "pkg:golang/github.com/google/go-cmp@v0.5.1",
        "pkg:golang/github.com/hashicorp/go-multierror@v1.1.0",
        "pkg:golang/github.com/keybase/go-keychain@v0.0.0-20200502122510-cda31fe0c86d",
        "pkg:golang/github.com/sirupsen/logrus@v1.7.0",
        "pkg:golang/github.com/stretchr/testify@v1.6.1",
        "pkg:golang/github.com/therecipe/qt@v0.0.0-20200701200531-7f61353ee73e",
        "pkg:golang/github.com/urfave/cli/v2@v2.2.0",
        "pkg:golang/github.com/vmihailenco/msgpack/v5@v5.1.3",
        "pkg:golang/golang.org/x/net@v0.0.0-20200707034311-ab3426394381"
      ]
    }
  ]
}
"""

In [ ]:
protonmail_graph = parse_cyclonedx_bom(json.loads(protonmail_bom))

In [ ]:
nx.draw(protonmail_graph, with_labels=True, node_color="#FFA600", edge_color="#FFA600")

## SPDX v2.x

SPDX stores its data in both the `package` and `file` objects.
Relationships are both explicitly listed in a `relationships` property and also spread out within a variety of other properties on various objects.
We have captured the `documentDescribes` and `hasFiles` relationships outside of the main `relationships` list but you may be interested in others.
The `parse_spdx2_package` and `parse_spdx2_file` functions can be modified to collect additional information about the objects.
Notice that we rename some of the node attributes so that they line up across object types.
This is to ensure that node attributes are not sparse.

In [ ]:
def parse_spdx2_package(package: dict) -> dict:
    return {
        "SPDXID": package.get("SPDXID", ""),
        "name": package.get("name", ""),
        "versionInfo": package.get("versionInfo", ""),
    }


def parse_spdx2_file(file: dict) -> dict:
    return {
        "SPDXID": file.get("SPDXID", ""),
        "name": file.get("fileName", ""),
        "versionInfo": "",
    }


def generate_spdx2_edges(nodes: list[tuple[int, dict]], bom: dict) -> list[tuple[int, int, dict]]:
    ref_lookup = {data["SPDXID"]: n for n, data in nodes if "SPDXID" in data}

    # spdx places some implicit relationships inside of property fields within the document
    # for example: the spdx document "describes" a software package (which should likely be listed in the packages section)
    edges = [
        ("0", ref_lookup[describee], {"relationshipType": "documentDescribes"})
        for describee in bom.get("documentDescribes", [])
        if describee in ref_lookup
    ]

    # another example of an implicit relationship as a property of an object
    for package in bom.get("packages", []):
        source = package.get("SPDXID")
        files = package.get("hasFiles", [])
        edges.extend(
            (ref_lookup[source], ref_lookup[target], {"relationshipType": "hasFiles"})
            for target in files
            if target in ref_lookup and source in ref_lookup
        )

    for relationship in bom.get("relationships", []):
        source = relationship.get("spdxElementId")
        target = relationship.get("relatedSpdxElement")
        if source in ref_lookup and target in ref_lookup:
            edges.append((ref_lookup[source], ref_lookup[target], relationship))

    return edges


def parse_spdx2_bom(bom: dict) -> nx.Graph:
    bom_document = {
        "SPDXID": bom.get("SPDXID", ""),
        "name": bom.get("name", ""),
        "versionInfo": bom.get("spdxVersion", ""),
    }
    packages = [parse_spdx2_package(p) for p in bom.get("packages", [])]
    files = [parse_spdx2_file(p) for p in bom.get("files", [])]

    next_id = -1
    nodes = [(str(next_id := next_id + 1), bom_document)]
    nodes.extend((str(next_id := next_id + 1), p) for p in packages)
    nodes.extend((str(next_id := next_id + 1), f) for f in files)

    edges = generate_spdx2_edges(nodes, bom)

    g = nx.Graph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)

    return g

In [ ]:
# copied from https://github.com/spdx/spdx-examples/blob/master/software/example8/spdx2.3/examplemaven-0.0.1.spdx.json
maven_example_bom = r"""
{
  "SPDXID" : "SPDXRef-DOCUMENT",
  "spdxVersion" : "SPDX-2.3",
  "creationInfo" : {
    "created" : "2022-10-23T15:44:16Z",
    "creators" : [ "Person: Gary O'Neall", "Tool: spdx-maven-plugin" ],
    "licenseListVersion" : "3.18"
  },
  "name" : "examplemaven",
  "dataLicense" : "CC0-1.0",
  "documentDescribes" : [ "SPDXRef-example" ],
  "documentNamespace" : "http://spdx.org/documents/examplemaven-0.0.1",
  "packages" : [ {
    "SPDXID" : "SPDXRef-junit",
    "copyrightText" : "UNSPECIFIED",
    "description" : "JUnit is a regression testing framework written by Erich Gamma and Kent Beck. It is used by the developer who implements unit tests in Java.",
    "downloadLocation" : "NOASSERTION",
    "filesAnalyzed" : false,
    "homepage" : "http://junit.org",
    "licenseConcluded" : "NOASSERTION",
    "licenseDeclared" : "CPL-1.0",
    "name" : "JUnit",
    "originator" : "Organization: JUnit",
    "summary" : "JUnit is a regression testing framework written by Erich Gamma and Kent Beck. It is used by the developer who implements unit tests in Java.",
    "versionInfo" : "3.8.1"
  }, {
    "SPDXID" : "SPDXRef-log4jslf4jbinding",
    "copyrightText" : "UNSPECIFIED",
    "description" : "The Apache Log4j SLF4J API binding to Log4j 2 Core",
    "downloadLocation" : "NOASSERTION",
    "filesAnalyzed" : false,
    "licenseConcluded" : "NOASSERTION",
    "licenseDeclared" : "NOASSERTION",
    "name" : "Apache Log4j SLF4J Binding",
    "summary" : "The Apache Log4j SLF4J API binding to Log4j 2 Core"
  }, {
    "SPDXID" : "SPDXRef-log4jslf4jApi",
    "copyrightText" : "UNSPECIFIED",
    "description" : "The slf4j API",
    "downloadLocation" : "NOASSERTION",
    "filesAnalyzed" : false,
    "homepage" : "http://www.slf4j.org",
    "licenseConcluded" : "NOASSERTION",
    "licenseDeclared" : "NOASSERTION",
    "name" : "SLF4J API Module",
    "summary" : "The slf4j API"
  }, {
    "SPDXID" : "SPDXRef-log4jApi",
    "copyrightText" : "UNSPECIFIED",
    "description" : "The Apache Log4j API",
    "downloadLocation" : "NOASSERTION",
    "filesAnalyzed" : false,
    "licenseConcluded" : "NOASSERTION",
    "licenseDeclared" : "NOASSERTION",
    "name" : "Apache Log4j API",
    "summary" : "The Apache Log4j API"
  }, {
    "SPDXID" : "SPDXRef-log4jImpl",
    "copyrightText" : "UNSPECIFIED",
    "description" : "The Apache Log4j Implementation",
    "downloadLocation" : "NOASSERTION",
    "filesAnalyzed" : false,
    "licenseConcluded" : "NOASSERTION",
    "licenseDeclared" : "NOASSERTION",
    "name" : "Apache Log4j Core",
    "summary" : "The Apache Log4j Implementation"
  }, {
    "SPDXID" : "SPDXRef-example",
    "checksums" : [ {
      "algorithm" : "SHA1",
      "checksumValue" : "b8a7e6c75001e6d78625cfc9a3103bf121abf8b4"
    } ],
    "copyrightText" : "Copyright (c) 2022 Source Auditor Inc.",
    "description" : "This is a simple example Maven project created using the Maven quickstart archetype with one dependency added.",
    "downloadLocation" : "NOASSERTION",
    "filesAnalyzed" : true,
    "homepage" : "https://github.com/spdx/spdx-examples",
    "licenseConcluded" : "Apache-2.0",
    "licenseDeclared" : "Apache-2.0",
    "licenseInfoFromFiles" : [ "Apache-2.0" ],
    "name" : "examplemaven",
    "originator" : "Organization: Linux Foundation",
    "packageFileName" : "examplemaven-0.0.1.jar",
    "packageVerificationCode" : {
      "packageVerificationCodeValue" : "c12417def36d7804096521de4280721e5863e68b"
    },
    "primaryPackagePurpose" : "LIBRARY",
    "hasFiles" : [ "SPDXRef-appsource", "SPDXRef-apptest" ],
    "summary" : "This is a simple example Maven project created using the Maven quickstart archetype with one dependency added.",
    "supplier" : "Organization: SPDX",
    "versionInfo" : "0.0.1"
  } ],
  "files" : [ {
    "SPDXID" : "SPDXRef-appsource",
    "checksums" : [ {
      "algorithm" : "SHA1",
      "checksumValue" : "a6f47dbc7e4615058490055172fe0065c55f8fc5"
    } ],
    "copyrightText" : "Copyright (c) 2020 Source Auditor Inc.",
    "fileContributors" : [ "Gary O'Neall" ],
    "fileName" : "./src/main/java/org/spdx/examplemaven/App.java",
    "fileTypes" : [ "SOURCE" ],
    "licenseComments" : "This file contains SPDX-License-Identifiers for Apache-2.0",
    "licenseConcluded" : "Apache-2.0",
    "licenseInfoInFiles" : [ "Apache-2.0" ],
    "noticeText" : "SPDX-License-Identifier: Apache-2.0\nCopyright (c) 2022 Source Auditor Inc."
  }, {
    "SPDXID" : "SPDXRef-apptest",
    "checksums" : [ {
      "algorithm" : "SHA1",
      "checksumValue" : "4b4df52d36588c8e9482d56eebc42336447f3dad"
    } ],
    "copyrightText" : "Copyright (c) 2020 Source Auditor Inc.",
    "fileContributors" : [ "Gary O'Neall" ],
    "fileName" : "./src/test/java/org/spdx/examplemaven/AppTest.java",
    "fileTypes" : [ "SOURCE" ],
    "licenseComments" : "This file contains SPDX-License-Identifiers for Apache-2.0",
    "licenseConcluded" : "Apache-2.0",
    "licenseInfoInFiles" : [ "Apache-2.0" ],
    "noticeText" : "SPDX-License-Identifier: Apache-2.0\nCopyright (c) 2022 Source Auditor Inc."
  } ],
  "relationships" : [ {
    "spdxElementId" : "SPDXRef-junit",
    "relationshipType" : "TEST_DEPENDENCY_OF",
    "relatedSpdxElement" : "SPDXRef-example",
    "comment" : "Relationship created based on Maven POM information"
  }, {
    "spdxElementId" : "SPDXRef-example",
    "relationshipType" : "DYNAMIC_LINK",
    "relatedSpdxElement" : "SPDXRef-log4jslf4jbinding",
    "comment" : "Relationship based on Maven POM file dependency information"
  }, {
    "spdxElementId" : "SPDXRef-example",
    "relationshipType" : "DYNAMIC_LINK",
    "relatedSpdxElement" : "SPDXRef-log4jslf4jApi",
    "comment" : "Relationship based on Maven POM file dependency information"
  }, {
    "spdxElementId" : "SPDXRef-example",
    "relationshipType" : "DYNAMIC_LINK",
    "relatedSpdxElement" : "SPDXRef-log4jApi",
    "comment" : "Relationship based on Maven POM file dependency information"
  }, {
    "spdxElementId" : "SPDXRef-example",
    "relationshipType" : "DYNAMIC_LINK",
    "relatedSpdxElement" : "SPDXRef-log4jImpl",
    "comment" : "Relationship based on Maven POM file dependency information"
  }, {
    "spdxElementId" : "SPDXRef-appsource",
    "relationshipType" : "GENERATES",
    "relatedSpdxElement" : "SPDXRef-example",
    "comment" : ""
  }, {
    "spdxElementId" : "SPDXRef-apptest",
    "relationshipType" : "TEST_CASE_OF",
    "relatedSpdxElement" : "SPDXRef-example",
    "comment" : ""
  } ]
}
"""

In [ ]:
maven_example_graph = parse_spdx2_bom(json.loads(maven_example_bom))

In [ ]:
nx.draw(maven_example_graph, with_labels=True, node_color="#FFA600", edge_color="#FFA600")